# 이미지 전처리 - 이미지 데이터 LMDB로 변환하기

In [ ]:
from torchvision.transform import Resize


# ResNet50 모델 구성

In [1]:
import tensorflow as tf
from tensorflow.keras import Model, regularizers
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, BatchNormalization, Activation, Lambda, add
from tensorflow.keras.losses import MeanSquaredError, Reduction
from tensorflow.optimizers import Adam

class ResNet50(): 
    _L2_WEIGHT_DECAY = 1e-4

    @staticmethod
    def _gen_l2_regularizer(use_l2_regularizer=True): 
        return regularizers.l2(ResNet50._L2_WEIGHT_DECAY) if use_l2_regularizer else None
    
    @staticmethod
    def _identity_block(input, filters, use_l2_regularizer): 
        filter1, filter2, filter3 = filters

        x = Conv2D(filters=filter1, kernel_size=1, strides=1, padding='same', activation=None, use_bias=False, kernel_initializer='he_normal', kernel_regularizer=ResNet50._gen_l2_regularizer(use_l2_regularizer), data_format='channels_last')(input)
        x = BatchNormalization(axis=0)(x)
        x = Activation('relu')(x)

        x = Conv2D(filters=filter2, kernel_size=3, strides=1, padding='same', activation=None, use_bias=False, kernel_initializer='he_normal', kernel_regularizer=ResNet50._gen_l2_regularizer(use_l2_regularizer), data_format='channels_last')(x)
        x = BatchNormalization(axis=0)(x)
        x = Activation('relu')(x)

        x = Conv2D(filters=filter3, kernel_size=1, strides=1, padding='same', activation=None, use_bias=False, kernel_initializer='he_normal', kernel_regularizer=ResNet50._gen_l2_regularizer(use_l2_regularizer), data_format='channels_last')(x)
        x = BatchNormalization(axis=0)(x)
        x = Activation('relu')(x)

        x = add([x, input])
        x = Activation('relu')(x)

        return x
    
    @staticmethod
    def _conv_block(input, filters, stride, use_l2_regularizer): 
        filter1, filter2, filter3 = filters

        x = Conv2D(filters=filter1, kernel_size=1, strides=1, padding='same', activaiton=None, use_bias=False, kernel_initializer='he_normal', kernel_regularizer=ResNet50._gen_l2_regularizer(use_l2_regularizer), data_format='channels_last')(input)
        x = BatchNormalization(axis=0)(x)
        x = Activation('relu')(x)

        x = Conv2D(filters=filter2, kernel_size=3, strides=stride, padding='same', activaiton=None, use_bias=False, kernel_initializer='he_normal', kernel_regularizer=ResNet50._gen_l2_regularizer(use_l2_regularizer), data_format='channels_last')(x)
        x = BatchNormalization(axis=0)(x)
        x = Activation('relu')(x)

        x = Conv2D(filters=filter3, kernel_size=1, strides=1, padding='same', activaiton=None, use_bias=False, kernel_initializer='he_normal', kernel_regularizer=ResNet50._gen_l2_regularizer(use_l2_regularizer), data_format='channels_last')(x)
        x = BatchNormalization(axis=0)(x)
        x = Activation('relu')(x)

        shortcut = Conv2D(filters=filter3, kernel_size=1, strides=stride, padding='same', activation=None, use_bias=False, kernel_initializer='he_normal', kernel_regularizer=ResNet50._gen_l2_regularizer(use_l2_regularizer), data_format='channels_last')(input)
        shortcut = BatchNormalization(axis=0)(shortcut)

        x = add([x, shortcut])
        x = Activation('relu')(x)

        return x
    
    def __init__(self, global_batch_size, img_size, number_regression_targets, learning_rate=3e-4, use_l2_regularizer=True):
        '''
        img_size needs to be (H, W, C) format
        '''


        self.img_size = img_size
        self.number_regression_targets = number_regression_targets
        self.learning_rate = learning_rate
        self.global_batch_size = global_batch_size
        self.use_l2_regularizer = use_l2_regularizer

        self.inputs = Input(shape=img_size)
        self.model = self._build_model()

        self.loss_fn = MeanSquaredError(reduction=Reduction.NONE)

        self.optimizer = Adam(learning_rate=self.learning_rate)

    def _build_model(self): 
        x = Conv2D(filters=64, kernel_size=7, strides=2, padding='same', activation=None, use_bias=False, kernel_initializer='he_normal', kernel_regularizer=ResNet50._gen_l2_regularizer(self.use_l2_regularizer), data_format='channels_last')(self.inputs)
        x = BatchNormalization(axis=0)(x)
        x = Activation('relu')(x)

        x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

        x = ResNet50._conv_block(x, [64, 64, 256], stride=1, use_l2_regularizer=self.use_l2_regularizer)
        x = ResNet50._identity_block(x, [64, 64, 256], use_l2_regularizer=self.use_l2_regularizer)
        x = ResNet50._identity_block(x, [64, 64, 256], use_l2_regularizer=self.use_l2_regularizer)

        x = ResNet50._conv_block(x, [128, 128, 512], stride=2, use_l2_regularizer=self.use_l2_regularizer)
        x = ResNet50._identity_block(x, [128, 128, 512], use_l2_regularizer=self.use_l2_regularizer)
        x = ResNet50._identity_block(x, [128, 128, 512], use_l2_regularizer=self.use_l2_regularizer)
        x = ResNet50._identity_block(x, [128, 128, 512], use_l2_regularizer=self.use_l2_regularizer)

        x = ResNet50._conv_block(x, [256, 256, 1024], stride=2, use_l2_regularizer=self.use_l2_regularizer)
        x = ResNet50._identity_block(x, [256, 256, 1024], use_l2_regularizer=self.use_l2_regularizer)
        x = ResNet50._identity_block(x, [256, 256, 1024], use_l2_regularizer=self.use_l2_regularizer)
        x = ResNet50._identity_block(x, [256, 256, 1024], use_l2_regularizer=self.use_l2_regularizer)
        x = ResNet50._identity_block(x, [256, 256, 1024], use_l2_regularizer=self.use_l2_regularizer)
        x = ResNet50._identity_block(x, [256, 256, 1024], use_l2_regularizer=self.use_l2_regularizer)

        x = ResNet50._conv_block(x, [512, 512, 2048], stride=2, use_l2_regularizer=self.use_l2_regularizer)
        x = ResNet50._identity_block(x, [512, 512, 2048], use_l2_regularizer=self.use_l2_regularizer)
        x = ResNet50._identity_block(x, [512, 512, 2048], use_l2_regularizer=self.use_l2_regularizer)

        rm_axes = [1, 2]
        x = Lambda(lambda x: tf.reduce_mean(x, rm_axes))(x)

        logits = Dense(self.number_regression_targets, kernel_initializer='he_normal', kernel_regularizer=ResNet50._gen_l2_regularizer(self.use_l2_regularizer), bias_regularizer=ResNet50._gen_l2_regularizer(self.use_l2_regularizer), activation=None, name='logits')(x)

        resnet50 = Model(self.inputs, logits, name='resnet50')

        return resnet50

    def get_keras_model(self): 
        return self.model
    
    def get_optimizer(self): 
        return self.optimizer
    
    def set_learning_rate(self, learning_rate): 
        self.optimizer.learning_rate = learning_rate

    def get_learning_rate(self): 
        return self.optimizer.learning_rate
    
    def train_step(self, inputs): 
        images, labels, loss_metric = inputs

        with tf.GradientTape() as tape: 
            logits = self.model(images, training=True)

            loss_value = self.loss_fn(labels, logits)
            loss_value = tf.reduce_sum(loss_value)

        grads = tape.gradient(loss_value, self.model.trainable_weights)

        self.optimizer.apply_gradients(zip(grads, self.model.trainable_weights))

        loss_metric.update_state(loss_value)

        return loss_value
    
    def test_step(self, inputs): 
        images, labels, loss_metric = inputs
        logits = self.model(images, training=False)

        loss_value = self.loss_fn(labels, logits)
        
        loss_value = tf.reduce_sum(loss_value, axis=0) / self.global_batch_size

        loss_metric.update_state(loss_value)

        return loss_value

    # Functions for Multi-GPU environment
    @tf.function
    def dist_train_step(self, dist_strategy, inputs): 
        per_gpu_loss = dist_strategy.experimental_run_v2(self.train_step, args=(inputs,))
        loss_value = dist_strategy.reduce(tf.distribute.ReduceOp.SUM, per_gpu_loss, axis=None)

        return loss_value

    @tf.function
    def dist_test_step(self, dist_strategy, inputs): 
        per_gpu_loss = dist_strategy.experimental_run_v2(self.test_step, args=(inputs,))
        loss_value = dist_strategy.reduce(tf.distribute.ReduceOp.SUM, per_gpu_loss, axis=None)

        return loss_value

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

import argparse
import datetime
import numpy as np
import time

global_batch_size=64
image_size=(250, 250)
number_outputs=2
train_epoch_size=20
test_epoch_size=20
train_log_dir = './train_log/'
test_log_dir = './test_log/'

if not os.path.exists(train_log_dir): os.makedirs(train_log_dir)
if not os.path.exists(test_log_dir): os.makedirs(test_log_dir)

resnet = ResNet50(global_batch_size, image_size, number_outputs)
checkpoint = tf.train.Checkpoint(optimizer=resnet.get_optimizer(), model=resnet.get_keras_model())

test_loss = []

train_loss_metric = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
test_loss_metric = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)

train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

epoch = 0
print('Running ResNet50')

while True: 
    print(f'---- Epoch: {epoch} ----')

    if epoch == 0: 
        curr_train_epoch_size = min(1000, train_epoch_size)